In [58]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs4
import pandas as pd
import lxml
import time

In [2]:
data = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")
soup = bs4(data.text)
standings_table = soup.select_one("table.stats_table")
st_links = standings_table.find_all("a")
st_hrefs = [link.get("href") for link in st_links]
st_hrefs =[l for l in st_hrefs if '/squads/' in l]
full_links = [f"https://fbref.com{l}" for l in st_hrefs]

In [36]:
url = full_links[0]
team_stats = requests.get(url)
team_soup = bs4(team_stats.text)
matches_tb = pd.read_html(team_stats.text, match="Scores & Fixtures")

all_links = team_soup.find_all('a')

all_links = [l.get("href") for l in all_links]

shoot_link = [l for l in all_links if l and 'all_comps/shooting/' in l]
matches_tb=matches_tb[0]
matches_tb.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2022-07-30,17:00,Community Shield,FA Community Shield,Sat,Neutral,L,1,3,Liverpool,NaN,NaN,57,NaN,Rúben Dias,4-3-3,Craig Pawson,Match Report,NaN
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,2.2,0.5,75,62443.0,İlkay Gündoğan,4-3-3,Michael Oliver,Match Report,NaN
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,1.7,0.1,67,53453.0,İlkay Gündoğan,4-2-3-1,David Coote,Match Report,NaN
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,2.1,1.8,69,52258.0,İlkay Gündoğan,4-3-3,Jarred Gillett,Match Report,NaN
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,2.2,0.1,74,53112.0,Kevin De Bruyne,4-2-3-1,Darren England,Match Report,NaN


In [42]:
shoot_url = requests.get(f"https://fbref.com{shoot_link[0]}");
shoot_tb = pd.read_html(shoot_url.text, match="Shooting")
shoot_tb[0].columns = shoot_tb[0].columns.droplevel()
shoot_tb=shoot_tb[0]
shoot_tb.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2022-07-30,17:00,Community Shield,FA Community Shield,Sat,Neutral,L,1,3,Liverpool,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,18.7,1.0,1,1,2.2,1.4,0.11,-0.2,-0.4,Match Report
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,17.5,0.0,0,0,1.7,1.7,0.09,1.3,1.3,Match Report
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,16.2,1.0,0,0,2.1,2.1,0.10,0.9,0.9,Match Report
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,14.1,0.0,0,0,2.2,2.2,0.13,1.8,1.8,Match Report


In [44]:
team_data = matches_tb.merge(shoot_tb[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
team_data.head()


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-07-30,17:00,Community Shield,FA Community Shield,Sat,Neutral,L,1,3,Liverpool,...,4-3-3,Craig Pawson,Match Report,NaN,14,8,NaN,NaN,0,0
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,4-3-3,Michael Oliver,Match Report,NaN,13,1,18.7,1.0,1,1
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,4-2-3-1,David Coote,Match Report,NaN,19,7,17.5,0.0,0,0
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,4-3-3,Jarred Gillett,Match Report,NaN,21,10,16.2,1.0,0,0
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,4-2-3-1,Darren England,Match Report,NaN,18,5,14.1,0.0,0,0
